In [1]:
import pandas as pd
import numpy as np

## Análise exploratória dos dados ##
 - Os dados foram obtidos no Kaggle, contém informações sobre a avaliação de posições de tabuleiros de xadrez: https://www.kaggle.com/datasets/ronakbadhe/chess-evaluations

Primeiramente, para começarmos a fazer uma análise exploratória para decidir os objetivos, devemos entender a estrutura dos dados com os quais estamos lidando.
Os dados estão divididos em 3 tabelas.
 - chessData contém informações sobre estados de tabuleiros na notação FEN, e a avaliação dele em centipawns (1/100 do valor de um peão)
 - random_evals contém diversos tabuleiros montados aleatoriamente a partir de diversas jogadas, e a avaliação dele em centipawns
 - tactic_evals é similar ao chessData, mas além disso, possui o melhor movimento possível naquele tabuleiro decidido com auxilio da ferramenta Linchess
#### Entender a notação FEN ####
 - A notação FEN é composta por 8 linhas, cada uma contendo peças, em maiusculo para brancas e minusculo para pretas, tendo um número de 1-8 indicando quantas casas vazias entre uma peça e outra. Após isso temos letras para ver a notação sobre de quem é o turno, possibilidade de roques e en passants
#### Centipawns #####
 - Outro fator comum entre todos os datasets é a avaliação do stockfish em centipawns, onde vão existir duas variações, as com # que indicam mate naquela posição, e as sem mate que indicam apenas a avaliação geral do tabuleiro
#### Move ####
 - A base de dados tactic_evals contém uma coluna Move, que contém informações da melhor jogada do Linchess, contendo a cordenada atual e a cordenada futura



In [4]:
chessData = pd.read_csv('Data/chessData.csv')
random_evals = pd.read_csv('Data/random_evals.csv')
tactic_evals = pd.read_csv('Data/tactic_evals.csv')

In [10]:
print(chessData.head())
print(random_evals.head())
print(tactic_evals.head())

                                                 FEN Evaluation
0  rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...        -10
1  rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...        +56
2  rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...         -9
3  rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...        +52
4  rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...        -26
                                                 FEN Evaluation
0  rnbqkb1r/pppppppp/B4n2/8/4P3/8/PPPP1PPP/RNBQK1...       -459
1  rnbqkb1r/pppppppp/5n2/1B6/4P3/8/PPPP1PPP/RNBQK...       -125
2  rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBN...       +198
3  rnbqkb1r/pppppppp/5n2/8/4P3/7N/PPPP1PPP/RNBQKB...       -155
4  rnbqkbnr/ppppp1pp/8/5p2/4P3/8/PPPP1PPP/RNBQKBN...       +209
                                                 FEN Evaluation  Move
0  r2qkbr1/pb1nn3/1ppp3p/8/3P1p2/2PB1N1P/PPQN1PP1...        #+2  d3g6
1  r2qkb2/pb1nn3/1ppp2rp/8/3P1p2/2P2N1P/PPQN1PP1/...        #+1  c2g6
2  r2qkbr1/pb1nn3/1ppp

In [ ]:
# Averiguando o comprimento das jogadas, é possível perceber que certos movimentos possuem 5 letras, para representar peões promovidos
dif_len_moves = tactic_evals[tactic_evals['Move'].str.len() != 4]
dif_len_moves

,FEN,Evaluation,Move,isCheck
573,6k1/p4p1p/2R5/2N2p2/3r4/1P6/P2p1PPP/4R1K1 b - ...,#-1,d2e1q,1
781,8/p7/R3P3/1pk3pp/8/P5PP/2p2PK1/8 b - - 1 47,-374,c2c1q,0
1259,8/p1rP4/1p4pp/5k2/8/P3KP1P/1P6/8 w - - 1 49,+509,d7d8q,0
1504,1k2r2r/1pp2P2/p7/2qpP2p/8/P7/2P2PPP/R4RK1 w - ...,-530,f7e8r,0
1564,2k5/P1P1b3/2K5/8/7p/7P/5p2/8 b - - 0 60,#+1,f2f1q,1
...,...,...,...,...
2627004,8/P2k4/2p5/6pp/1Pn5/8/8/6K1 w - - 0 42,+484,a7a8q,0
2627426,8/6P1/8/6K1/8/8/3p2Q1/2k1r3 b - - 0 75,0,d2d1q,0
2627427,8/6P1/8/6K1/8/8/6Q1/2kqr3 w - - 0 76,0,g7g8q,0
2627498,8/1P2b3/2K1p3/p3p3/P3Pk2/2p5/8/8 w - - 0 77,+455,b7b8q,0


## Pré-processamento dos dados ##

Formato de um código FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1

Para utilizar esses dados em um modelo de IA, precisamos fazer algums tratamentos para transformar eles em um input

#### Pré-processamento código FEN ####

- Tokenização: Podemos análisar o código mais facilmente tokenizando o valor para os valores presentes em suas linhas

- Remoção de dados irrelevantes: Para nosso objetivo, que não envolve validação de jogadas, as informações após o tabuleiro não são úteis

- Converter números para 0: Transformar um número naquela quantidade de 0's é importante para representar as casas vazias

- Conversão para matriz: Utilizando a pontuação de sarrat para converter peças em números e transformar o código em uma matriz 8x8

#### Pré-processamento evaluation ####

 - Cheques: Para transformar esses dados em algo mais legível, os cheques serão tratados para existir uma colunas apenas com valores boleanos, se aquele tabuleiro esta com mate ou não, mantendo a coluna de evaluation separada

#### Pré-processamento jogada (tactic_evals) ####

- Tokenização: Similarmente a como fizemos com a notação FEN, para as jogadas, iremos separar os valores em tokens por letra

- Tratar peça extra: Jogadas que envolvem promoções de peões, possuem um algarismo a mais denotando a peça para a qual ele foi promovido, portanto, irei realizar a substituição dessa peça, pela pontuação de sarrat equivalente, e acrecentar um valor 0 ao final dos tokens que não possuem ela para manter as entradas no mesmo formato sem ter perca de informação

- Conversão tokens: Para simplificar o entendimento pelo modelo, não seria um problema para a notação, substituir o valor das letras, por seu equivalente numérico, que na prática é a mesma coisa nesse contexto 



In [1]:
sarrat = {
    'P':1,
    'N':3.1,
    'B':3.3,
    'R':5,
    'Q':7.9,
    'K':2.2,
    'p':-1,
    'n':-3.1,
    'b':-3.3,
    'r':-5,
    'q':-7.9,
    'k':-2.2
}

In [3]:
import re
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Operações para tratar boards
def separate_board(board):
    return board.apply(lambda x: x.replace('/', ' ').split())

def remove_extras(board):
    board_novo = []
    for i in board:
        board_novo.append(board[0:8])
    return board_novo

def num_to_zeros(board):
    return [re.sub(r'[1-9]', '0', row) for row in board]
            
def fen_to_matrix(fen):
    matrix = []
    for line in fen:
        row = []
        for letter in line:
            row.append(sarrat.get(f'{letter}', letter))
        matrix.append(row)
    return matrix

#Operações para tratar movimentos

def separate_move(move):
    return list(move)

#Operações para evaluation

def mate_column(evaluation):
    return ['#' in eval for eval in evaluation]

def evaluate_column(evaluation):
    return 


In [ ]:
pipeline_transform_fen = Pipeline([
    ('Separate board into lines', FunctionTransformer(separate_board)),
    ('Remove no-board info', FunctionTransformer(remove_extras)),
    ('Convert num to zeros', FunctionTransformer(num_to_zeros)),
    ('Convert fen notation to matrix', FunctionTransformer(fen_to_matrix))
])

In [ ]:
#from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Concatenate, Input, Conv2D
#from tensorflow.keras.models import Model
#sfrom tensorflow.keras.callbacks import EarlyStopping